In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import cv2
import numpy as np

In [3]:
#Define ROI for cropping the frames
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    match_mask_color = 255 
    
    cv2.fillPoly(mask, vertices, match_mask_color) #create a polygon mask in the vertices area
    masked_image = cv2.bitwise_and(img, mask) #Apply and operation between image and mask (area of the image that is black in mask, will be black (zeo))
    return masked_image

In [4]:
region_of_interest_vertices = [[0, 330], [170, 70], [400,70], [500, 330]] #Custom ROI

In [5]:
cap = cv2.VideoCapture('/content/drive/MyDrive/road.webm')


frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
frame_rate=int(cap.get(5)) #FPS


out = cv2.VideoWriter('/content/drive/MyDrive/road_segmented.mp4',cv2.VideoWriter_fourcc(*'XVID'),frame_rate, (frame_width,frame_height))

In [6]:
while cap.isOpened():
    ret, img= cap.read()
    
    if ret:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        cropped_image = region_of_interest(gray,np.array([region_of_interest_vertices], np.int32))
            
            
        blurred = cv2.GaussianBlur(cropped_image, (3, 3), 0)
        filterr=cv2.threshold(blurred,150,255,cv2.THRESH_BINARY)[1]
        
        
        canny_edges = cv2.Canny(filterr,200,255)
        
        cv2.imshow("segmented",canny_edges)
        cv2.imshow("roi",cropped_image)
        out.write(cv2.cvtColor(canny_edges, cv2.COLOR_GRAY2BGR))
        if cv2.waitKey(0) & 0xFF == ord('q'):
                break
    else:
         break
        
cap.release()
out.release()